In [6]:
from gurobipy import *

# Store demand in thousands of units
demand = [15, 18, 24, 20]

# Distribution_Centre capacity in thousands of units
capacity = [20, 28, 25, 31, 24]

# Fixed costs for each Distribution_Centre
fixedCosts = [12000, 15000, 17000, 13000, 16000]

# Transportation costs per thousand units
transCosts = [[4000, 2000, 3000, 2500, 4500],
              [2500, 2600, 3400, 3000, 4000],
              [1200, 1800, 2600, 4100, 3000],
              [2200, 2600, 3100, 3700, 3200]
              
             ]

# Range of Distribution_Centres and Stores
Distribution_Centres = range(len(capacity))
Stores = range(len(demand))

# Model
m = Model("facility")

# Distribution_Centre open decision variables: open[p] == 1 if Distribution_Centre p is open.
open = m.addVars(Distribution_Centres,
                 vtype=GRB.BINARY,
                 obj=fixedCosts,
                 name="open")

# Transportation decision variables: transport[w,p] captures the
# optimal quantity to transport to Store w from Distribution_Centre p
transport = m.addVars(Stores, Distribution_Centres, obj=transCosts, name="trans")

# You could use Python looping constructs and m.addVar() to create
# these decision variables instead.  The following would be equivalent
# to the preceding two statements...
#
#open = []
#for p in Distribution_Centres:
#  open.append(m.addVar(vtype=GRB.BINARY,
#                       obj=fixedCosts[p],
#                       name="open[%d]" % p))
#
#transport = []
#for w in Stores:
#  transport.append([])
#  for p in Distribution_Centres:
#    transport[w].append(m.addVar(obj=transCosts[w][p],
#                                 name="trans[%d,%d]" % (w, p)))

# The objective is to minimize the total fixed and variable costs
m.modelSense = GRB.MINIMIZE

# Production constraints
# Note that the right-hand limit sets the production to zero if the Distribution_Centre
# is closed
m.addConstrs(
    (transport.sum('*',p) <= capacity[p]*open[p] for p in Distribution_Centres),
    "Capacity")

# Using Python looping constructs, the preceding would be...
#
#for p in Distribution_Centres:
#  m.addConstr(sum(transport[w][p] for w in Stores) <= capacity[p] * open[p],
#              "Capacity[%d]" % p)

# Demand constraints
m.addConstrs(
    (transport.sum(w) == demand[w] for w in Stores),
    "Demand")

# ... and the preceding would be ...
#for w in Stores:
#  m.addConstr(sum(transport[w][p] for p in Distribution_Centres) == demand[w], "Demand[%d]" % w)

# Guess at the starting point: close the Distribution_Centre with the highest fixed costs;
# open all others

# First, open all Distribution_Centres
for p in Distribution_Centres:
    open[p].start = 1.0

# Now close the Distribution_Centre with the highest fixed cost
print('Initial guess:')
maxFixed = max(fixedCosts)
for p in Distribution_Centres:
    if fixedCosts[p] == maxFixed:
        open[p].start = 0.0
        print('Closing Distribution_Centre %s' % p)
        break
print('')

# Use barrier to solve root relaxation
m.Params.method = 2

# Solve
m.optimize()

# Print solution
print('\nTOTAL COSTS: %g' % m.objVal)
print('SOLUTION:')
for p in Distribution_Centres:
    if open[p].x > 0.99:
        print('Distribution_Centre %s open' % p)
        for w in Stores:
            if transport[w,p].x > 0:
                print('  Transport %g units to Store %s' % \
                      (transport[w,p].x, w))
    else:
        print('Distribution_Centre %s closed!' % p)

Initial guess:
Closing Distribution_Centre 2

Changed value of parameter method to 2
   Prev: -1  Min: -1  Max: 5  Default: -1
Optimize a model with 9 rows, 25 columns and 45 nonzeros
Variable types: 20 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+03, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]

Loaded MIP start with objective 228700

Presolve time: 0.01s
Presolved: 9 rows, 25 columns, 45 nonzeros
Variable types: 20 continuous, 5 integer (5 binary)
Presolved: 9 rows, 25 columns, 45 nonzeros

Root barrier log...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.000e+01
 Factor NZ  : 4.500e+01
 Factor Ops : 2.850e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   8.61381939e+05 -1.75744130e+05  7.00e+00 5.19e+03  2.67e+04     0s
   1   2.4967131